<a href="https://colab.research.google.com/github/azhgh22/Walmart-Recruiting-Store-Sales-Forecasting/blob/main/notebooks/n_beats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import userdata
token = userdata.get('GITHUB_TOKEN')
user_name = userdata.get('GITHUB_USERNAME')
mail = userdata.get('GITHUB_MAIL')

!git config --global user.name "{user_name}"
!git config --global user.email "{mail}"
!git clone https://{token}@github.com/azhgh22/Walmart-Recruiting-Store-Sales-Forecasting.git

%cd Walmart-Recruiting-Store-Sales-Forecasting

from google.colab import userdata
! pip install -r ./requirements.txt
kaggle_json_path = userdata.get('KAGGLE_JSON_PATH')
! ./src/data_loader.sh -f {kaggle_json_path}

Mounted at /content/drive
Cloning into 'Walmart-Recruiting-Store-Sales-Forecasting'...
remote: Enumerating objects: 334, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 334 (delta 65), reused 48 (delta 21), pack-reused 209 (from 1)
Receiving objects: 100% (334/334), 6.90 MiB | 24.12 MiB/s, done.
Resolving deltas: 100% (162/162), done.
/content/Walmart-Recruiting-Store-Sales-Forecasting
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.8/285.8 kB 18.

In [3]:
# **Torch**

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin
import torch
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS
from neuralforecast.losses.pytorch import MSE

from src.config import *

stores = pd.read_csv(STORES_PATH)
features = pd.read_csv(FEATURES_PATH)
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

train['Date'] = pd.to_datetime(train.Date)
# test['Date'] = pd.to_datetime(test.Date)

In [4]:
from src.processing import split_data
x_train,y_train,x_val,y_val = split_data(train)

**Cross Validation**

In [11]:
class CustomNBEATS(NBEATS):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.optimizer = torch.optim.AdamW(self.parameters(), lr=1e-3)

    def set_optim(self,optimizer):
      self.optimizer = optimizer

    def configure_optimizers(self):
        # Define your custom optimizer here
        optimizer = self.optimizer #torch.optim.AdamW(self.parameters(), lr=1e-3)

        # Optional: add scheduler if needed
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

        return {
            'optimizer': optimizer,
            # 'lr_scheduler': {
            #     'scheduler': scheduler,
            #     'interval': 'epoch',
            #     'frequency': 1
            # }
        }

In [12]:
from sklearn.base import BaseEstimator, TransformerMixin

class Preprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        x_copy = X.copy()
        if 'y' not in x_copy.columns:
            x_copy['y'] = 0
        x_copy['Date'] = pd.to_datetime(x_copy['Date'])
        return x_copy


In [13]:
class NBEATSModel(BaseEstimator, TransformerMixin):
    def __init__(self,models:list[NBEATS]=None):
        if models == None or len(models)==0:
          if torch.cuda.is_available():
            device = 'cuda'
          else:
            device = 'cpu'
          models = [CustomNBEATS(
            input_size=15,
            h=48,
            max_steps=25 * 104,
            batch_size=64,
            stack_types=(['identity', 'trend', 'seasonality']),
            n_blocks = [1,1,1],
            random_seed=42,
            accelerator=device,
            devices=1,
            logger=True,
            enable_progress_bar=False,
            enable_model_summary=False,
        )]
        self.nf = NeuralForecast(models=models, freq='W-FRI')

    def fit(self, X, y=None):
        df = self.create_data_nbeats(X,y)
        df.sort_values(by=['ds'],inplace=True)
        self.nf.fit(df)

        return self

    def transform(self, X):
        preds = self.nf.predict()

        real = self.create_data_nbeats(X,X.y)
        merged = pd.merge(real, preds, on=['unique_id','ds'],how='left')
        # print(merged.shape)
        merged.fillna(0,inplace=True)


        return merged


    def predict(self,X=None):
        return self.transform(X)

    def create_data_nbeats(self,x:pd.DataFrame,y:pd.DataFrame) -> pd.DataFrame:
        df = pd.DataFrame()

        df = df.reset_index()
        df["Date"] = x.Date
        df["Store"] = x.Store
        df["Dept"] = x.Dept
        df["IsHoliday"] = x.IsHoliday.astype(int)
        df.rename(columns={"Date": "ds"}, inplace=True)
        df["unique_id"] = df["Store"].astype(str) + "_" + df["Dept"].astype(str)
        df["y"] = y.values
        df = df[["unique_id", "ds", "y",'IsHoliday']].copy()
        return df

In [14]:
from os import pipe
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('preprocess', Preprocessor()),
    ('model', NBEATSModel())
])

model = pipeline.fit(x_train,y_train)

INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [15]:
from src.utils import wmae
x_val['y'] = y_val
preds = model.predict(x_val)
wmae(preds['y'],preds['CustomNBEATS'],preds['IsHoliday'])

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


np.float64(2462.0331187605916)

# **Grid Search**

In [28]:
l = [1e-6,1e-5,1e-4,1e-3,1e-2,1e-1,0,1,10]
for lr in l:
  estim = CustomNBEATS(
              input_size=15,
              h=48,
              max_steps=25 * 104,
              batch_size=64,
              stack_types=(['identity', 'trend', 'seasonality']+['identity']*8),
              n_blocks = [1,1,1]+[1]*8,
              random_seed=42,
              accelerator='gpu',
              devices=1,
              logger=True,
              enable_progress_bar=False,
              enable_model_summary=False,)

  estim.set_optim(torch.optim.AdamW(estim.parameters(), lr=1e-3,weight_decay=lr))

  estimators = [estim]

  pipeline = Pipeline([
      ('preprocess', Preprocessor()),
      ('model', NBEATSModel(estimators))
  ])

  model = pipeline.fit(x_train,y_train)
  x_val['y'] = y_val
  preds = model.predict(x_val)
  score = wmae(preds['y'],preds['CustomNBEATS'],preds['IsHoliday'])
  print(lr,score)

INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=2600` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transf

1e-06 2392.3769091145205


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=2600` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 i

1e-05 2392.3769091145205


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=2600` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 i

...

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=2600` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 i

1 2523.376480050695


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=2600` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


10 8963.528303510586


/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [54]:
estim = CustomNBEATS(
            input_size=15,
            h=48,
            max_steps=25 * 104,
            batch_size=64,
            stack_types=(['identity', 'trend', 'seasonality']+['identity']*8),
            n_blocks = [1,1,1]+[1]*8,
            random_seed=42,
            accelerator='gpu',
            devices=1,
            logger=True,
            enable_progress_bar=False,
            enable_model_summary=False,)

estim.set_optim(torch.optim.AdamW(estim.parameters(), lr=1e-3,weight_decay=0))

estimators = [estim]

pipeline = Pipeline([
    ('preprocess', Preprocessor()),
    ('model', NBEATSModel(estimators))
])

# model = pipeline.fit(x_train,y_train)
# x_val['y'] = y_val
# preds = model.predict(x_val)
# score = wmae(preds['y'],preds['CustomNBEATS'],preds['IsHoliday'])
# print(lr,score)

INFO:lightning_fabric.utilities.seed:Seed set to 42


In [31]:
! wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: azhgh22 (MLBeasts) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False
...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False
421566,45,98,2012-10-05,628.10,False
421567,45,98,2012-10-12,1061.02,False
421568,45,98,2012-10-19,760.01,False


In [57]:
import wandb
import joblib


fin_model = pipeline.fit(train.drop(columns=['Weekly_Sales'].copy()),train['Weekly_Sales'].copy())

joblib.dump(fin_model, "nbeats_run1.pkl")
wandb.init(project="Walmart Recruiting - Store Sales Forecasting", name="nbeats:run1")

wandb.config.update({
    'score_metric' : 'WMAE',
    'score_policy' : {
        'weight on holidays' : 5,
        'weight on non_holidays' : 1
    },
    'model' : 'nbeats',
    'learning_rate' : 0.001,
    'weight_decay' : 0,
    'batch_size' : 64,
    'max_depth' : 7,
    'max_steps' : 25 * 104,
    'input_size' : 15,
    'horizon': 48,
    'architecture' : ['identity', 'trend', 'seasonality']+['identity'],
    'n_blocks' : [1,1,1]+[1]*8,
    'random_state': 42,
    'objective' : 'reg:squarederror',
})

wandb.log({
    'val_wmae': score
})


artifact = wandb.Artifact(
    name="nbeats_run1",
    type="model",
)

artifact.add_file("nbeats_run1.pkl")
wandb.log_artifact(artifact)

wandb.finish()

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=2600` reached.


val_wmae,▁
val_wmae,2392.37691


In [61]:
pred = fin_model.predict(test)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [63]:
test1 = test.copy()
test1['unique_id'] = test1["Store"].astype(str) + "_" + test1["Dept"].astype(str)
test1['ds'] = pd.to_datetime(test1['Date'])

In [68]:
merged = pd.merge(test1, pred, on=['unique_id','ds'],how='left')
merged['Id'] = test1["unique_id"].astype(str) + "_" + test1["ds"].astype(str)
merged['Weekly_Sales'] = merged['CustomNBEATS']

In [70]:
merged[['Id','Weekly_Sales']].to_csv('submission.csv', index=False)